In [42]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [43]:
import pandas as pd
train_data = pd.read_csv('../Dataset/cleaned_train.csv', encoding='utf-8')
test_data = pd.read_csv('../Dataset/cleaned_dev.csv', encoding='utf-8')

In [44]:
# from preprocessing import clean_data
# train_data = clean_data(train_data)
# test_data = clean_data(test_data)
# # Save the data to a csv file
# train_data.to_csv('../Dataset/cleaned_train.csv', index=False)
# test_data.to_csv('../Dataset/cleaned_dev.csv', index=False)

In [45]:
print(train_data['text'].head(5))

0    بيل غيتس يتلقى لقاح تصوير الابرة السيرنجة الدو...
1    وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...
2    قولكن رح يكونو اد المسؤولية لبنان يوصل اللقاح ...
3    وزير الصحة فخر الدين قوجة يتلقى جرعة لقاح كورو...
4    وئام وهاب يشتم الدول الخليجية طلة اعلامية ويتس...
Name: text, dtype: object


In [46]:
from sklearn.model_selection import train_test_split
# Unpack the data into text, category and stance
Train_X, Test_X, cat_Train_Y, cat_Test_Y, stance_Train_Y, stance_Test_Y = train_data['text'], test_data['text'], train_data['category'], test_data['category'], train_data['stance'], test_data['stance']


In [47]:
bert_encode_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
bert_preprocess_url='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [48]:
bert_proprocess=hub.KerasLayer(bert_preprocess_url)
bert_encoder=hub.KerasLayer(bert_encode_url)

In [ ]:
def get_sentence_embeddings(sentences):
    preprocessed_text=bert_proprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
# get the sentence embedding for the sentences
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# preprocessing the data using bert preprocess layer
preprocessed_text=bert_proprocess(text_input)

# get the bert encoder layer
outputs = bert_encoder(preprocessed_text)

# Nueral network layer
l=tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# construct final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [50]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [52]:
# Split each tweet into words
Train_X = Train_X.apply(lambda x: x.split())

In [53]:
print(Train_X.head(5))

0    [بيل, غيتس, يتلقى, لقاح, تصوير, الابرة, السيرن...
1    [وزير, الصحة, لحد, اليوم, وتحديدا, هلأ, بمؤتمر...
2    [قولكن, رح, يكونو, اد, المسؤولية, لبنان, يوصل,...
3    [وزير, الصحة, فخر, الدين, قوجة, يتلقى, جرعة, ل...
4    [وئام, وهاب, يشتم, الدول, الخليجية, طلة, اعلام...
Name: text, dtype: object


In [54]:
METRICS=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=METRICS)
history = model.fit(Train_X,stance_Train_Y,epochs=5)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [23]:
model.evaluate(Test_X,Test_Y)

55/55 [==============================] - 301s 5s/step - loss: 0.5832 - accuracy: 0.7928 - precision: 0.8552 - recall: 1.0000


[0.5831922292709351, 0.7927876114845276, 0.8551803231239319, 1.0]

In [ ]:
# y_predict = model.predict(Test_X)
# y_predicted = y_predict.flatten()